In [1]:
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding
import datasets
from datasets import load_metric
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import sys
import json

In [2]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds.predictions, eval_preds.label_ids
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def save2Json(metrics, filename):
    json_object = json.dumps(metrics, indent=4)
    with open(filename, "w") as outfile:
        outfile.write(json_object)


def run_benchmark_single(model_name, tokenizer_checkpoint, model_checkpoint, SPARSE_PERCENT):
    raw_datasets = datasets.load_dataset('glue', 'mrpc')
    
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
    def tokenize_function(sample):
        return tokenizer(sample['sentence1'], sample['sentence2'], truncation=True)
    tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

    training_args = TrainingArguments(
        num_train_epochs = 1,
        output_dir=f'output/glue-{model_name}_{SPARSE_PERCENT}/',
    )

    trainer = Trainer(
        model,
        training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator, 
        tokenizer=tokenizer,
        # compute_metrics=compute_metrics
    )
    print(f"Start training {model_checkpoint}...")

    trains = trainer.train()
    # predictions = trainer.predict(tokenized_datasets['validation'])
    save2Json(trains.metrics, filename=f'output/glue-{model_name}_{SPARSE_PERCENT}.json')


In [3]:
run_benchmark_single(
    model_name="gpt2", 
    tokenizer_checkpoint='./models/gpt2-glue-tokenizer/', 
    model_checkpoint="./models/gpt2-glue_0.1/", 
    SPARSE_PERCENT=10
)

Found cached dataset glue (/Users/shuyuzhou/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

The following columns in the training set don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1. If idx, sentence2, sentence1 are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
/opt/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3668
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 459
  Number of trainable parameters = 124441344


Start training ./models/gpt2-glue_0.1/...


  0%|          | 0/459 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 1958.2886, 'train_samples_per_second': 1.873, 'train_steps_per_second': 0.234, 'train_loss': 0.6446202762246391, 'epoch': 1.0}
